In [1]:
# Storage SC

In [2]:
import logging

import smartpynector as sp
from utils import *

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("StorageSC")

In [4]:
# Constants
# GraphDB endpoints
READ_URL = "https://graphdb.odissei.nl/repositories/MateuszTest"
WRITE_URL = "https://graphdb.odissei.nl/repositories/MateuszTest/statements"

THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
# prefixes of ontologies used in the graph patterns
PREFIXES = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "saref": "https://w3id.org/saref#",
    "xsd": "http://www.w3.org/2001/XMLSchema#",
    "ex": "http://example.org/",
}
# measurement graph pattern
MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                        ?meas saref:hasValue ?temp .
                        ?meas saref:isMeasuredIn saref:TemperatureUnit .
                        ?meas saref:hasTimestamp ?timestamp .
                        ?meas saref:isMeasurementOf ?room_id .
                        ?meas saref:relatesToProperty saref:Temperature .
                        ?meas saref:measurementMadeBy ?device_id ."""

# graph pattern describing a timeseries of measurements with timestamp between ?startTimestamp and ?endTimestamp
H_MEAS_II_GRAPH_PATTERN = """?timeseries rdf:type ex:Timeseries .
                            ?timeseries ex:hasMeasurement ?meas .
                            ?timeseries ex:measuredAfter ?startTimestamp .
                            ?timeseries ex:measuredBefore ?endTimestamp .
                            ?meas rdf:type saref:Measurement .
                            ?meas saref:hasValue ?temp .
                            ?meas saref:hasTimestamp ?timestamp .
                            """

In [5]:
# parsing results of the querry to match the ask binding
def parse_response(response, start_timestamp, end_timestamp, timeseries):
    variables = response["head"]["vars"]
    bindings = response["results"]["bindings"]
    result = []
    for binding in bindings:
        measurement = {}
        for var in variables:
            # we add double quotes "<var> "to each string of each variable (necessary to be compatible with the Knowledge Engine
            measurement[var] = f'"{binding[var]["value"]}"'
        # Here we add timeseries, startTimestamp, endTimestamp to each measurement so that it matches the ask binding
        measurement["timeseries"] = timeseries
        measurement["startTimestamp"] = start_timestamp
        measurement["endTimestamp"] = end_timestamp

        result.append(measurement)
    return result

In [6]:
# Return measurements in datetime range from GraphDB
def handle_answer_measurements(query_bindings):
    for binding in query_bindings:
        logger.info(f"Handling answer for: {binding['timeseries']}")

        start_timestamp = binding["startTimestamp"]
        end_timestamp = binding["endTimestamp"]
        timeseries = binding["timeseries"]
        # This sparqle querry returns all measurements with timestamp between start_timestamp and end_timestamp
        sparql_query = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                            PREFIX saref: <https://w3id.org/saref#>
                            PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

                            SELECT *

                            WHERE {{
                                ?meas rdf:type saref:Measurement .
                                ?meas saref:hasValue ?temp .
                                ?meas saref:isMeasuredIn saref:TemperatureUnit .
                                ?meas saref:hasTimestamp ?timestamp .
                                FILTER (xsd:dateTime(?timestamp) >= {start_timestamp}^^xsd:dateTime && xsd:dateTime(?timestamp) <= {end_timestamp}^^xsd:dateTime)
                                }}"""
        # running the sparql querry against GraphDB
        res = sp.run_sparql_query(READ_URL, sparql_query)

        # parsing results of the querry to match the ask binding
        result = parse_response(res, start_timestamp, end_timestamp, timeseries)

    return result


def handle_react_measurements(bindings):
    # storing the live measurements in GraphDB
    sp.store_data_in_graphdb(
        graph_pattern=MEAS_GRAPH_PATTERN,
        binding_set=bindings,
        prefixes=PREFIXES,
        read_url=READ_URL,
        write_url=WRITE_URL,
    )
    for binding in bindings:
        logger.info(f"Saving measurement {binding['meas']}")
    return []

In [7]:
def start_storage_kb(kb_id, kb_name, kb_description, ke_endpoint):
    # delete in case allready exists
    delete_knowledge_base(kb_id, ke_endpoint)

    # register kb
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)

    # register answer ki
    answer_measurements_ki = register_answer_knowledge_interaction(
        H_MEAS_II_GRAPH_PATTERN,
        "answer-historical-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    # register react ki
    react_measurements_ki = register_react_knowledge_interaction(
        MEAS_GRAPH_PATTERN,
        None,
        "react-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

    # start handle loop for handling answer and react kis
    start_handle_loop(
        {
            answer_measurements_ki: handle_answer_measurements,
            react_measurements_ki: handle_react_measurements,
        },
        kb_id,
        ke_endpoint,
    )

In [8]:
start_storage_kb(
    "http://example.org/storage",
    "Storage",
    "GraphDB smart connector",
    "http://knowledge_engine:8280/rest/",
)

2023-06-22 00:47:56 INFO {"messageType":"error","message":"Deletion of knowledge base failed, because it could not be found."}
2023-06-22 00:47:56 INFO registered Storage
2023-06-22 00:47:56 INFO received issued knowledge interaction id: http://example.org/storage/interaction/answer-historical-measurements
2023-06-22 00:47:57 INFO received issued knowledge interaction id: http://example.org/storage/interaction/react-measurements
2023-06-22 00:48:01 INFO Saving measurement <http://0.0.0.0:8001/thermostat/measurements/36125660-658f-44f7-9146-9a0133021e3f>
2023-06-22 00:48:07 INFO Saving measurement <http://0.0.0.0:8001/thermostat/measurements/3957d941-f961-4524-a240-fb42e4f1a8fd>
2023-06-22 00:48:13 INFO Saving measurement <http://0.0.0.0:8001/thermostat/measurements/c954290e-ccfe-4fb7-b6d9-b10f32dc9c38>
2023-06-22 00:48:19 INFO Saving measurement <http://0.0.0.0:8001/thermostat/measurements/eadf97c1-fede-463a-9786-c2d9d077870c>
2023-06-22 00:48:19 INFO Handling answer for: <http://0.0.0

KeyboardInterrupt: 